# Topic Modeling with Gensim (Python)

1. Set up Pyspark in Colab
2. Install nltk, Download nltk stopwords
3. Stemming and Lemmatization
    * Install Gensim
    * Install spaCy
    * Install other dependencies
    * Download and import speeches
4. LDA analysis
    * Data Cleaning
    * Bigrams and Trigrams
    * Topic Modeling Inputs
    * Optimal Number of Topics
    * Viewing LDA topics
5. LDA Evaluation
    * Model Perplexity and Coherence
    * Install pyLDAvis
    * Install LDA Mallet
6. Interpreting LDA Results
    * Dominant topic in each sentence
    * Most representative document for each topic
    * Topic distribution across documents
7. Conclusions
    * Strengths of Topic Modeling
    * Words of caution



## Set up Pyspark in Colab

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system.
Follow the steps to install the dependencies:

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
# Can take as much as 6 minutes!

!wget -qN https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.1-bin-hadoop3.2"

In [4]:
!pip install -q findspark

## 2. Install nltk, download nltk stopwords

Definition: high-frequency words like *the, to, and, also* that we sometimes want to filter out of a document before further processing.


In [5]:
# Run in python console
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Let's import the stopwords and make them available in `stop_words`.

In [6]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# Additional words we would like to ignore, for example, for a corpus of emails
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
print(len(stop_words), stop_words)

179 ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than

## 3. Stemming and lemmatization

For grammatical reasons, documents are going to use different forms of a word, such as organize, organizes, and organizing. Additionally, there are families of derivationally related words with similar meanings, such as democracy, democratic, and democratization. In many situations, it seems as if it would be useful for a search for one of these words to return documents that contain another word in the set.

The goal of both stemming and lemmatization is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form.

For grammatical reasons, documents are going to use different forms of a word, such as organize, organizes, and organizing. Additionally, there are families of derivationally related words with similar meanings, such as democracy, democratic, and democratization. In many situations, it seems as if it would be useful for a search for one of these words to return documents that contain another word in the set.

The goal of both stemming and lemmatization is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form. For instance:

**am, are, is $\Rightarrow$ be**

**car, cars, car's, cars' $\Rightarrow$ car**

The result of this mapping of text will be something like:

**the boy's cars are different colors $\Rightarrow$ the boy car be differ color**

* *Stemming* usually refers to a crude heuristic process that chops off the ends of words in the hope of achieving this goal correctly most of the time, and often includes the removal of derivational affixes. For example, **saw** $\Rightarrow$ **s**.

* *Lemmatization* usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma . For example, **saw**  ⇒ **see** or **saw**  ⇒ **saw** depending on whether the use of the token was as a verb or a noun.

Stemming is fast and crude whereas lemmatization commonly only collapses the different inflectional forms of a lemma.

Stemming or lemmatization is often done by an additional plug-in component to the indexing process, and a number of such components exist, both commercial and open-source.

Source: [Stanford NLP](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html)

In [7]:
# installing other dependencies
import pandas as pd
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)
import numpy as np

import re
import string
import pprint

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 4. Import data

Downloads and prepares a dictionary with

* Keys = speech_name, e.g., `1789-Washington.txt` and
* Value = List of list of sentences, e.g., [<br/>['Fellow', '-', 'Citizens', 'of', 'the', 'Senate', 'and', 'of', 'the', 'House', 'of', 'Representatives', ':'], <br/>['Among', 'the', 'vicissitudes', 'incident', 'to', 'life', 'no', 'event', 'could', 'have', 'filled', 'me', 'with', 'greater', 'anxieties', 'than', 'that', 'of', 'which', 'the', 'notification', 'was', 'transmitted', 'by', 'your', 'order', ',', 'and', 'received', 'on', 'the', '14th', 'day', 'of', 'the', 'present', 'month', '.'], <br/>...]

In [8]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:

!ls
import pandas as pd
reviews_df = pd.read_csv('/content/drive/MyDrive/data/reviews.csv')  # Update the path

# Extract relevant text column
reviews_text = reviews_df['Review Text']



drive  sample_data  spark-3.2.1-bin-hadoop3.2  spark-3.2.1-bin-hadoop3.2.tgz


### 3. Install Gensim and spaCy model (for lemmatization)
Lemmatization is nothing but converting a word to its root word. For example: the lemma of the word ‘machines’ is ‘machine’. Likewise, ‘walking’ –> ‘walk’, ‘mice’ –> ‘mouse’ and so on.

In [15]:
!pip install gensim
!python3 -m spacy download en_core_web_sm

2023-11-21 22:49:29.676736: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-21 22:49:29.676824: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-21 22:49:29.676875: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-21 22:49:29.693049: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 22:49:31.192341: W tensorflow/compiler/

In [27]:
# Gensim

import gensim
from gensim import corpora

from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
spacy.load('en_core_web_sm')

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 8. Tokenize words and Clean-up text </h2>
</div>

The sentences look better now, but you want to tokenize each sentence into a list of words, removing punctuations and unneccessary characters altogether.

Gensim's `simple_preprocess()` is great for this. Additionally I have set `deacc=True` to remove the punctuations.

In [33]:
from nltk.tokenize import word_tokenize


def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence).encode('utf-8'), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(reviews_text))
data_words = [dw for dw in data_words if len(dw)>0]
print(data_words[3401:3406])



[['love', 'the', 'swing', 'style', 'boxy', 'fit', 'versatile', 'works', 'well', 'over', 'jeans', 'or', 'any', 'pants', 'and', 'top', 'like', 'the', 'coated', 'linen', 'or', 'cotton', 'look', 'serves', 'much', 'the', 'same', 'purpose', 'as', 'any', 'light', 'weight', 'or', 'denim', 'jacket', 'good', 'to', 'wear', 'over', 'top', 'to', 'complete', 'an', 'outfit', 'or', 'as', 'an', 'extra', 'thin', 'layer', 'but', 'not', 'substantial', 'outdoor', 'jacket', 'really', 'quite', 'cute', 'fairly', 'tts', 'but', 'if', 'between', 'sizes', 'size', 'down'], ['the', 'top', 'was', 'very', 'small', 'am', 'usually', 'size', 'small', 'but', 'this', 'was', 'tight', 'around', 'the', 'top', 'arm', 'holes', 'and', 'it', 'was', 'short', 'also', 'didn', 'notice', 'if', 'it', 'said', 'it', 'was', 'sheer', 'see', 'through', 'in', 'the', 'description', 'but', 'it', 'definitely', 'requires', 'another', 'shirt', 'underneath'], ['ve', 'had', 'this', 'jacket', 'for', 'few', 'weeks', 'and', 'it', 'has', 'been', 'terr

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 9. Create Bigram and Trigram Models</h2>
</div>

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words that occur frequently.

Some examples in our example are: 'soviet union', etc.

Gensim's `Phrases` model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to `Phrases` are `min_count` and `threshold`. The higher the values of these param, the harder it is for words to be combined to bigrams.

In [36]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['absolutely', 'wonderful', 'silky', 'and', 'sexy', 'and', 'comfortable']


<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 10. Remove Stopwords, Make Bigrams and Lemmatize</h2>
</div>


The bigrams model is ready. Let's define the functions to remove the stopwords, make bigrams and lemmatization and call them sequentially.

In [38]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Let’s call the functions in order.

In [39]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
print(data_words_nostops)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
print(data_words_bigrams)

# In the end, we didn't create trigrams. Should have taken the extra time.

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[['absolutely', 'wonderful', 'silky', 'sexy', 'comfortable']]


In [ ]:
# pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 11. Create the Dictionary and Corpus needed for Topic Modeling</h2>
</div>

The two main inputs to the LDA topic model are the dictionary(`id2word`) and the corpus. Let's create them.

In [40]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
# Human readable format of corpus (term-frequency)
# There is nothing magical about 2500:2510, I just wanted to examine a random location
print ([[(id2word[id], freq) for id, freq in cp] for cp in corpus[2500:2512]])

[[('dress', 3), ('fit', 1), ('really', 1), ('somewhat', 1), ('work', 2), ('return', 1), ('look', 1), ('big', 2), ('nice', 1), ('material', 1), ('right', 1), ('chest', 1), ('large', 1), ('figure', 1), ('hope', 1), ('shape', 1), ('way', 1), ('think', 1), ('outfit', 1), ('cheste', 1), ('especially', 1), ('give', 1), ('unfortunately', 1), ('band', 1), ('consider', 1), ('staple', 1), ('wayyy', 1)], [('comfortable', 1), ('dress', 1), ('find', 1), ('fit', 2), ('size', 1), ('small', 1), ('time', 1), ('wear', 2), ('well', 2), ('long', 1), ('light', 1), ('cup', 1), ('fabric', 1), ('feel', 1), ('large', 1), ('open', 1), ('breast', 1), ('quite', 1), ('tend', 1), ('girl', 2), ('complaint', 1), ('refreshing', 1), ('suggest', 1), ('wind', 1), ('delightful', 1), ('clasp', 1), ('fly', 1), ('blow', 1), ('numerous', 1)], [('wonderful', 1), ('dress', 3), ('store', 1), ('nicely', 1), ('size', 1), ('great', 1), ('wear', 1), ('legging', 1), ('color', 1), ('style', 1), ('person', 1), ('nice', 1), ('drape', 1)

Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).

For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.

This is used as the input by the LDA model.


If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.

Or, you can see a human readable form of the corpus itself.

In [ ]:
speeches_corpus = dict(id2word)

We are ready to build the topic model.

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 12. Building the Topic Model</h2>
</div>

We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, `alpha` and `beta` are hyperparameters that affect sparsity of the topics. According to the gensim docs, both default to 1.0/num_topics prior.

`chunksize` is the number of documents to be used in each training chunk.  `update_every` determines how often the model parameters should be updated and `passes` is the total number of training passes.

The next cell can take about 5 minutes to process.

In [41]:
# Build LDA model
num_topics = 10
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 13. View the topics in LDA model</h2>
</div>

The above LDA model is built with 5 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

You can see the keywords for each topic and the weightage(importance) of each keyword using `lda_model.print_topics()` as shown next.

In [44]:
# Print the Keywords in the topics
## pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [45]:
!pip install pyLDAvis==3.2.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.1-py2.py3-none-any.whl size=136162 sha256=e036816a1880ada3d49b53838923e31593c0561c0a9798ab38505213ef752571
  Stored in directory: /root/.cache/pip/wheels/d3/35/9c/c084bb1c3a4b8a7feb32d57c88f572dcf5c3accc949478893f
Successfully built pyLDAvis


In [46]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

In [47]:
print ([itm for itm in dir(doc_lda) if not itm.startswith('__')])
print ([itm for itm in dir(doc_lda.obj) if (not itm.startswith('__')) and (not itm.startswith('_'))])
doc_lda.obj.print_topics(num_topics=-1)

['_adapt_by_suffix', '_load_specials', '_save_specials', '_smart_save', 'add_lifecycle_event', 'chunksize', 'corpus', 'load', 'metadata', 'obj', 'save', 'save_corpus']
['add_lifecycle_event', 'alpha', 'bound', 'callbacks', 'chunksize', 'clear', 'decay', 'diff', 'dispatcher', 'distributed', 'do_estep', 'do_mstep', 'dtype', 'eta', 'eval_every', 'expElogbeta', 'gamma_threshold', 'get_document_topics', 'get_term_topics', 'get_topic_terms', 'get_topics', 'id2word', 'inference', 'init_dir_prior', 'iterations', 'lifecycle_events', 'load', 'log_perplexity', 'minimum_phi_value', 'minimum_probability', 'num_terms', 'num_topics', 'num_updates', 'numworkers', 'offset', 'optimize_alpha', 'optimize_eta', 'passes', 'per_word_topics', 'print_topic', 'print_topics', 'random_state', 'save', 'show_topic', 'show_topics', 'state', 'sync_state', 'top_topics', 'update', 'update_alpha', 'update_eta', 'update_every']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.478*"dress" + 0.027*"knee" + 0.024*"lace" + 0.019*"easily" + 0.014*"curve" + 0.012*"wedding" + 0.012*"cool" + 0.010*"substantial" + 0.010*"wrinkle" + 0.010*"mean"'),
 (1,
  '0.214*"sweater" + 0.064*"wide" + 0.041*"coat" + 0.040*"disappoint" + 0.039*"inch" + 0.030*"slim" + 0.024*"new" + 0.022*"staple" + 0.021*"suit" + 0.018*"couple"'),
 (2,
  '0.091*"skirt" + 0.083*"jean" + 0.057*"pair" + 0.036*"casual" + 0.030*"wash" + 0.027*"comfy" + 0.025*"add" + 0.025*"warm" + 0.016*"heel" + 0.016*"neckline"'),
 (3,
  '0.057*"time" + 0.047*"first" + 0.045*"many" + 0.043*"compliment" + 0.041*"receive" + 0.031*"know" + 0.024*"embroidery" + 0.023*"knit" + 0.023*"adorable" + 0.019*"feminine"'),
 (4,
  '0.042*"love" + 0.038*"wear" + 0.033*"top" + 0.028*"color" + 0.025*"great" + 0.021*"fabric" + 0.020*"get" + 0.018*"buy" + 0.018*"look" + 0.016*"little"'),
 (5,
  '0.068*"actually" + 0.062*"glad" + 0.042*"issue" + 0.039*"cardigan" + 0.032*"seam" + 0.031*"due" + 0.031*"texture" + 0.028*"clothing" +

How to interpret this?

|Topic | representation
:-----:|:--------------:|
0 | '0.291*"thank" + 0.075*"make" + 0.059*"century" + 0.052*"strength" + 0.050*"first" + 0.017*"freedom" + 0.007*"do" + 0.001*"high" |
1 | '0.080*"take" + 0.005*"ceremony"
2 | 0.202*"time" + 0.000*"troop"
3 | ???
4 | 0.067*"land" + 0.011*"principle"
5 | 0.225*"people" + 0.022*"use"
6 | 0.336*"nation"
7 | 0.022*"protect" + 0.021*"bless" + 0.015*"story" + 0.013*"day" + 0.012*"world" + 0.011*"stand" + 0.011*"child" + 0.010*"let" + 0.010*"democracy" + 0.009*"know"
8 | ???
9 | 0.179*"today" + 0.158*"great" + 0.082*"year" + 0.017*"government" + 0.000*"troop"

<hr/>

## 14. Visualize the topics-keywords

Now that the LDA model is built, the next step is to examine the produced topics and the associated keywords. There is no better tool than pyLDAvis package’s interactive chart and is designed to work well with jupyter notebooks.

Keep in mind that [Topic 1 in pyLDAvis will be Topic 0 from gensim](
https://stackoverflow.com/a/62228381/653651).

In [48]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.340048 -0.005329       1        1  43.912433
9      0.335314 -0.004820       2        1  30.531664
7     -0.062712 -0.253809       3        1   5.150155
2     -0.060939  0.390952       4        1   4.849363
0     -0.059613 -0.024164       5        1   4.484804
3     -0.069015 -0.026319       6        1   4.171569
1     -0.098682 -0.020284       7        1   2.157317
5     -0.105690 -0.019441       8        1   1.786140
6     -0.108188 -0.018613       9        1   1.632478
8     -0.110524 -0.018173      10        1   1.324077, topic_info=           Term          Freq         Total Category  logprob  loglift
7         dress  12148.000000  12148.000000  Default  30.0000  30.0000
48         size  10887.000000  10887.000000  Default  29.0000  29.0000
30          fit  10530.000000  10530.000000  Default  28.0000  28.0000
352     sweater   2614.000000   2614.000000  Default  27.0000  27.0000
15         love  10475.000000  10475.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
1996        lie     90.955433     91.845532  Topic10  -4.4127   4.3147
290     provide     86.959333     87.849439  Topic10  -4.4576   4.3143
1030  exception     84.428217     85.318312  Topic10  -4.4872   4.3140
1211     overly     80.898838     81.788914  Topic10  -4.5299   4.3135
976      modest     80.313171     81.203245  Topic10  -4.5371   4.3134

[348 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
506       8  0.998553    actually
507       4  0.998136         add
1220     10  0.996517    addition
67       10  0.995176  adjustable
491       6  0.997602    adorable
...     ...       ...         ...
55        2  0.023949        work
1097      9  0.993271       worry
857       5  0.995732     wrinkle
1297      7  0.988057       young
57        6  0.998167      zipper

[334 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 10, 8, 3, 1, 4, 2, 6, 7, 9])

## 15. How to interpret pyLDAvis’s output?

Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.

A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.


## 16. Compute Model Perplexity and Coherence Score

Model perplexity and [topic coherence](https://rare-technologies.com/what-is-topic-coherence/) provide a convenient measure to judge how good a given topic model is. In my experience, topic coherence score, in particular, has been more helpful.

In [49]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -6.841313739927716

Coherence Score:  0.3057667247711376


Those were the topics for the chosen LDA model.

### All Rights Reserved.

This notebook is proprietary content of machinelearningplus.com. This can be shared solely for educational purposes, with due credits to machinelearningplus.com

The original notebook was published by **Selva Prabhakaran** [online](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/).

There is an additional segment in Prabhakan's notebook that builds an LDA Mallet model. It has fallen prey to obsolescence &mdash; I really should take the time to fix it.